In [100]:
import imp
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

import os
import numpy as np
import pandas as pd
import pickle
from backtesting import Backtest, Strategy
from backtesting.lib import resample_apply
import matplotlib.pyplot as plt
import pandas_datareader.data as web
import datetime
from decimal import Decimal
from dateutil import relativedelta
import importlib
import BacktestingStrategies.Optimization as opt

start = datetime.datetime.today() + relativedelta.relativedelta(years=-1)
end = datetime.date.today()
# data = web.DataReader('1570.T', 'yahoo', start, end)  #NEXT FUNDS 日経平均レバレッジ・インデックス連動型上場投信
data = web.DataReader('1357.T', 'yahoo', start, end)  #NEXT FUNDS 日経平均ダブルインバース・インデックス連動型上場投信

data = data.astype("double")

initial = 1000_000
fname = f"html/plot-"
df_returns = pd.DataFrame()

In [110]:
def execute_backtest(bt):
    output = bt.run()
    print(output)
    display(output._trades)
    bt.plot(filename = fname + str(output._strategy) + ".html") # 実行結果（グラフ）
    df_temp = pd.DataFrame(output).transpose().drop(["_equity_curve", "_trades"], axis=1)
    global df_returns
    df_returns = pd.concat([df_returns, df_temp], ignore_index=True)
    display(output._equity_curve)

def execute_optimize(bt, opt_params):
    output = bt.optimize(**opt_params)
    print(output)
    display(output._trades)
    display(output._strategy)
    bt.plot(filename = fname + str(output._strategy) + "_optimize.html")
    df_temp = pd.DataFrame(output).transpose().drop(["_equity_curve", "_trades"], axis=1)
    df_temp["_strategy"] = str(output._strategy) + "_optimize"
    global df_returns
    df_returns = pd.concat([df_returns, df_temp], ignore_index=True)

In [111]:
import BacktestingStrategies.Strategy_Momentum as btst
importlib.reload(btst)

# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    btst.Momentum, # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True, #自動でポジションをクローズ(オープン)
)
execute_backtest(bt)

# opt_params = {
#     "small_threshold": range(1, 10, 1),\
#     "large_threshold": range(1, 10, 1),\
#     "maximize": "Equity Final [$]",\
#     "method": 'grid',\
#     "constraint": lambda x: (x.large_threshold > x.small_threshold),\
#     "max_tries": 1000
# }
# execute_optimize(bt, opt_params)

# opt_params = {
#     "small_threshold": range(1, 10, 1),\
#     "large_threshold": range(1, 10, 1),\
#     "period_long": range(1, 10, 1),\
#     "period_short": range(1, 10, 1),\
#     "maximize": "Profit Factor",\
#     "method": 'grid',\
#     "constraint": lambda x: (x.large_threshold > x.small_threshold) & \
#         (x.period_long > x.period_short),\
#     "max_tries": 1000
# }
# execute_optimize(bt, opt_params)

Start                     2021-10-11 00:00:00
End                       2022-10-11 00:00:00
Duration                    365 days 00:00:00
Exposure Time [%]                    78.77551
Equity Final [$]                 671311.40505
Equity Peak [$]                  1297627.5816
Return [%]                         -32.868859
Buy & Hold Return [%]               -3.931204
Return (Ann.) [%]                  -33.628903
Volatility (Ann.) [%]               26.470132
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -50.818369
Avg. Drawdown [%]                  -10.986106
Max. Drawdown Duration      216 days 00:00:00
Avg. Drawdown Duration       38 days 00:00:00
# Trades                                    7
Win Rate [%]                        14.285714
Best Trade [%]                       1.559138
Worst Trade [%]                    -15.626113
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,2564,40,109,389.92060,396.0,15587.58160,0.015591,2021-12-08,2022-03-23,105 days
1,-2577,110,144,394.03980,416.0,-56591.43540,-0.055731,2022-03-24,2022-05-17,54 days
2,2344,145,154,409.01465,391.0,-42226.33960,-0.044044,2022-05-18,2022-05-31,13 days
3,-2476,159,169,370.15860,428.0,-143215.30640,-0.156261,2022-06-07,2022-06-21,14 days
4,1859,170,184,416.04930,403.0,-24258.64870,-0.031365,2022-06-22,2022-07-12,20 days
5,-2018,193,235,371.15365,396.0,-50139.93430,-0.066944,2022-07-26,2022-09-27,63 days
6,1743,236,244,400.97505,385.0,-27844.51215,-0.039841,2022-09-28,2022-10-11,13 days


,Equity,DrawdownPct,DrawdownDuration
2021-10-11,1000000.00000,0.000000,NaT
2021-10-12,1000000.00000,0.000000,NaT
2021-10-13,1000000.00000,0.000000,NaT
2021-10-14,1000000.00000,0.000000,NaT
2021-10-15,1000000.00000,0.000000,NaT
...,...,...,...
2022-10-04,653881.40505,0.496095,NaT
2022-10-05,648652.40505,0.500124,NaT
2022-10-06,638194.40505,0.508184,NaT
2022-10-07,650395.40505,0.498781,NaT


青がTraining期間
<br>
オレンジが検証（Forwardテスト）期間
<br>
![](2022-10-11-16-08-08.png)

In [103]:
#warmup ... Training期間で指標が最初からあるようにするために必要なデータ
#lookback ... Trainingに使うデータ
#validation ... Forward testに使うデータ
def walk_forward(
        strategy,
        data_full,
        warmup_bars,
        lookback_bars,
        validation_bars,
        cash = 10_000_000,
        commission = 0.002
    ):

    stats_master = []

    #endが全データから検証期間をマイナスしたところ
    #step幅が検証期間
    loop = 1
    # print("len(data_full)" + str(len(data_full)))
    # print("warmup_bars" + str(warmup_bars))
    # print("lookback_bars" + str(lookback_bars))
    # print("validation_bars" + str(validation_bars))

    for i in range(lookback_bars + warmup_bars, len(data_full) - validation_bars, validation_bars):
        # print("loop" + str(loop))
        # loop = loop + 1
        # print("i" + str(i))

        #普通のFor文だとstartがTraining期間の終わりになってしまうのでその分マイナス
        training_data = data_full.iloc[i - lookback_bars - warmup_bars : i]
        # training_data = data_full.iloc[i - lookback_bars - warmup_bars : i]
        forwardtest_data = data_full.iloc[i - warmup_bars  : i + validation_bars]
        # forwardtest_data = data_full.iloc[training_data  : len(validation_bars)]

        # print("training_data")
        # print(training_data)
        # print("forwardtest_data")
        # print(forwardtest_data)

        bt_training = Backtest(training_data, strategy, cash = cash, commission=commission)
        stats_training = bt.optimize(
                            small_threshold = range(1, 10, 1),
                            large_threshold = range(1, 10, 1),
                            maximize = "Equity Final [$]",
                            method = 'grid',
                            constraint = lambda x: (x.large_threshold > x.small_threshold),
                            max_tries = 1000
                            )

        small_threshold = stats_training._strategy.small_threshold
        large_threshold = stats_training._strategy.large_threshold
        # period_short = stats_training._strategy.period_short
        # period_long = stats_training._strategy.period_long

        bt_validation = Backtest(forwardtest_data, strategy, cash = cash, commission=commission)
        stats_validation = bt_validation.run(
                                small_threshold = small_threshold,
                                large_threshold = large_threshold)

        stats_master.append(stats_validation)

    return stats_master

**Do Forward test**

In [104]:
#学習期間と検証期間の割合は1:3くらい
lookback_bars = 66
validation_bars = 22
warmup_bars = 39

import BacktestingStrategies.Strategy_Momentum as strtg
importlib.reload(strtg)

print(data)

stats = walk_forward(
            strtg.Momentum,
            data,
            lookback_bars=lookback_bars,
            validation_bars=validation_bars,
            warmup_bars=warmup_bars)

             High    Low   Open  Close       Volume  Adj Close
2021-10-11  425.0  404.0  422.0  407.0   87120033.0      407.0
2021-10-12  417.0  407.0  409.0  416.0   75496853.0      416.0
2021-10-13  422.0  410.0  420.0  416.0   50498103.0      416.0
2021-10-14  415.0  403.0  413.0  404.0   43973512.0      404.0
2021-10-15  400.0  390.0  396.0  390.0   68652441.0      390.0
...           ...    ...    ...    ...          ...        ...
2022-10-04  387.0  375.0  386.0  375.0  114901364.0      375.0
2022-10-05  375.0  370.0  371.0  372.0   70916112.0      372.0
2022-10-06  372.0  364.0  371.0  366.0   85440215.0      366.0
2022-10-07  377.0  370.0  376.0  373.0   95441240.0      373.0
2022-10-11  393.0  383.0  385.0  391.0  124572635.0      391.0

[245 rows x 6 columns]


**Plots**

Plot stats of a particular result

In [105]:
def plot_stats(data_full, stats):
    equity_curve = stats._equity_curve
    aligned_data = data_full.reindex(equity_curve.index)

    bt = Backtest(aligned_data, strtg.Momentum, cash = 1000_000, commission = 0.002)
    bt.plot(results = stats)

In [109]:
strategy_returns = [x["Return [%]"] for x in stats]
hold_returns = [x["Buy & Hold Return [%]"] for x in stats]

import statistics
print(statistics.mean(strategy_returns))
print(statistics.mean(hold_returns))

-2.0749129166666695
-2.9042925375683697
